### 第10章の章末演習問題
※ ここではGoogle Colaraboratoryでの実行を想定しています。

※ Google Colaraboratoryでbashコマンドを実行するには、命令の前に!をつけます。

## [0] 必要な準備

In [ ]:
# 必須ライブラリのインストール
!pip install SimpleITK
!pip install diskcache
!pip install cassandra-driver

# 関連スクリプトの取得
!mkdir util
!wget https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/p2ch10/dsets.py
!wget -P ./util https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/util/disk.py
!wget -P ./util https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/util/logconf.py
!wget -P ./util https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/util/util.py

# 原著のGitHub上のコードではエラーになるため、ライブラリのインポート部分を微修正
# 参考Issue：https://github.com/deep-learning-with-pytorch/dlwpt-code/issues/27
!sed -i -e '3,4d' ./util/disk.py
!sed -i '1ifrom diskcache.core import MODE_BINARY' ./util/disk.py
!sed -i '1ifrom io import BytesIO' ./util/disk.py
!sed -i '1ifrom diskcache.core import io' ./util/disk.py
!sed -i '1ifrom diskcache import FanoutCache, Disk,core' ./util/disk.py
!sed -i '1ifrom cassandra.cqltypes import BytesType' ./util/disk.py
!head ./util/disk.py

     |████████████████████████████████| 44.9MB 91kB/s 
     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 3.8MB 3.2MB/s 
--2020-11-03 11:02:52--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/p2ch10/dsets.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6929 (6.8K) [text/plain]
Saving to: ‘dsets.py’

dsets.py            100%[===================>]   6.77K  --.-KB/s    in 0s      

2020-11-03 11:02:52 (31.8 MB/s) - ‘dsets.py’ saved [6929/6929]

--2020-11-03 11:02:52--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/util/disk.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting t

In [ ]:
# データのダウンロード
!wget -P ./data/part2/luna https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/annotations.csv
!wget -P ./data/part2/luna https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/candidates.csv


--2020-11-03 11:02:54--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/annotations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135799 (133K) [text/plain]
Saving to: ‘./data/part2/luna/annotations.csv’

annotations.csv     100%[===================>] 132.62K  --.-KB/s    in 0.03s   

2020-11-03 11:02:54 (3.99 MB/s) - ‘./data/part2/luna/annotations.csv’ saved [135799/135799]

--2020-11-03 11:02:54--  https://raw.githubusercontent.com/deep-learning-with-pytorch/dlwpt-code/master/data/part2/luna/candidates.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443

In [ ]:
# データのダウンロード（15分ほどかかります）と解凍（5分ほどかかります）
# ※Google Colaboratoryではときおり、ダウンロードが途中でリトライになる可能性があります

!wget -P ./data-unversioned/part2/luna https://zenodo.org/record/3723295/files/subset0.zip
%cd /content/data-unversioned/part2/luna
!jar xvf subset0.zip
%cd /content


--2020-11-03 11:02:56--  https://zenodo.org/record/3723295/files/subset0.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6811924508 (6.3G) [application/octet-stream]
Saving to: ‘./data-unversioned/part2/luna/subset0.zip’

subset0.zip         100%[===================>]   6.34G  14.1MB/s    in 12m 24s 

2020-11-03 11:15:21 (8.73 MB/s) - ‘./data-unversioned/part2/luna/subset0.zip’ saved [6811924508/6811924508]

/content/data-unversioned/part2/luna
 inflated: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.mhd
 inflated: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260.raw
 inflated: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492.mhd
 inflated: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492.raw
 inflated: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.1090025255245

## [1] LunaDatasetインスタンスを反復するプログラムを実装し、実行にどの程度時間が掛かるか計ってください。なお、時間の都合上、最初の N=1000 個のサンプルを反復するように制限するオプションを付す判断が合理的かもしれません。

In [ ]:
from dsets import LunaDataset
dataset = LunaDataset()

2020-11-03 10:49:38,720 INFO     pid:108 dsets:173:__init__ <dsets.LunaDataset object at 0x7fdb50c55828>: 56938 training samples


### （a）初回の実行でどの程度時間が掛かりましたか。

In [ ]:
# 回答（まずは1000個で試してみましょう）

In [ ]:
# 回答　Wall time: 3min 56s
%%time

for i in range(1000):
    print(dataset[i])


ストリーミング出力は最後の 5000 行に切り捨てられました。
          [  66.,  107.,   96.,  ...,   48.,   13.,   15.],
          [  73.,  104.,  135.,  ...,   59.,   39.,   13.],
          ...,
          [-783., -783., -806.,  ..., -857., -846., -804.],
          [-770., -831., -843.,  ..., -870., -858., -857.],
          [-855., -878., -836.,  ..., -854., -869., -867.]],

         [[  77.,   80.,  108.,  ...,   32.,   58.,   50.],
          [  47.,  129.,  145.,  ...,   23.,   -9.,    8.],
          [  92.,  128.,  107.,  ...,   74.,    1.,   12.],
          ...,
          [-839., -839., -843.,  ..., -827., -818., -805.],
          [-798., -768., -798.,  ..., -842., -875., -862.],
          [-808., -800., -831.,  ..., -840., -868., -862.]],

         [[  86.,   88.,  104.,  ...,   -9.,   32.,   84.],
          [  83.,   71.,   61.,  ...,   45.,   76.,   70.],
          [  50.,   80.,  104.,  ...,  111.,   71.,   36.],
          ...,
          [-810., -737., -785.,  ..., -816., -835., -863.],
          [-833., -

### （b）2回目の実行でどの程度時間が掛かりましたか。

In [ ]:
# 回答（まずは1000個で試してみましょう）

In [ ]:
# 回答　Wall time: 7.03 s
%%time

for i in range(1000):
    print(dataset[i])
# （a）でgetCtRawCandidate()関数の結果がキャッシュされているため、実行時間が著しく短くなる。

ストリーミング出力は最後の 5000 行に切り捨てられました。
          [  66.,  107.,   96.,  ...,   48.,   13.,   15.],
          [  73.,  104.,  135.,  ...,   59.,   39.,   13.],
          ...,
          [-783., -783., -806.,  ..., -857., -846., -804.],
          [-770., -831., -843.,  ..., -870., -858., -857.],
          [-855., -878., -836.,  ..., -854., -869., -867.]],

         [[  77.,   80.,  108.,  ...,   32.,   58.,   50.],
          [  47.,  129.,  145.,  ...,   23.,   -9.,    8.],
          [  92.,  128.,  107.,  ...,   74.,    1.,   12.],
          ...,
          [-839., -839., -843.,  ..., -827., -818., -805.],
          [-798., -768., -798.,  ..., -842., -875., -862.],
          [-808., -800., -831.,  ..., -840., -868., -862.]],

         [[  86.,   88.,  104.,  ...,   -9.,   32.,   84.],
          [  83.,   71.,   61.,  ...,   45.,   76.,   70.],
          [  50.,   80.,  104.,  ...,  111.,   71.,   36.],
          ...,
          [-810., -737., -785.,  ..., -816., -835., -863.],
          [-833., -

### （c）キャッシュをクリアした場合、実行時間にどのような影響がありますか？

In [ ]:
# 回答（まずは1000個で試してみましょう）

In [ ]:
# キャッシュ内容のフォルダを削除
!rm -rf data-unversioned/cache

In [ ]:
# 回答　Wall time: 2min 25s
# ディスクキャッシュは消え、メモリキャッシュが残っているため、(a)と(b)の間の結果になった
%%time

for i in range(1000):
    print(dataset[i])


ストリーミング出力は最後の 5000 行に切り捨てられました。
          [  66.,  107.,   96.,  ...,   48.,   13.,   15.],
          [  73.,  104.,  135.,  ...,   59.,   39.,   13.],
          ...,
          [-783., -783., -806.,  ..., -857., -846., -804.],
          [-770., -831., -843.,  ..., -870., -858., -857.],
          [-855., -878., -836.,  ..., -854., -869., -867.]],

         [[  77.,   80.,  108.,  ...,   32.,   58.,   50.],
          [  47.,  129.,  145.,  ...,   23.,   -9.,    8.],
          [  92.,  128.,  107.,  ...,   74.,    1.,   12.],
          ...,
          [-839., -839., -843.,  ..., -827., -818., -805.],
          [-798., -768., -798.,  ..., -842., -875., -862.],
          [-808., -800., -831.,  ..., -840., -868., -862.]],

         [[  86.,   88.,  104.,  ...,   -9.,   32.,   84.],
          [  83.,   71.,   61.,  ...,   45.,   76.,   70.],
          [  50.,   80.,  104.,  ...,  111.,   71.,   36.],
          ...,
          [-810., -737., -785.,  ..., -816., -835., -863.],
          [-833., -

### （d） 最後のN=1000 個のサンプルを使用した場合、最初、または2回目の実行時間にどのような影響がありますか。 

In [ ]:
# 回答

In [ ]:
%%time
# 最初の実行　Wall time: 1.24 s

print(dataset[-1000])  # 後ろから1000個目を使用

# getCtRawCandidate()関数の結果はキャッシュされていないため、初回まずまず時間がかかる

(tensor([[[[ 113.,  103.,  103.,  ...,   94.,  106.,  104.],
          [  89.,   71.,   89.,  ...,   87.,   84.,   72.],
          [  90.,   65.,   58.,  ...,   68.,   68.,   93.],
          ...,
          [ 137.,  156.,  133.,  ..., -102., -113., -115.],
          [ 113.,  115.,  116.,  ..., -113.,  -93.,  -81.],
          [ 110.,  100.,  102.,  ..., -122.,  -73.,  -61.]],

         [[ 115.,  117.,   94.,  ...,   85.,   71.,   74.],
          [  76.,   71.,   80.,  ...,   56.,   51.,   58.],
          [  84.,   86.,   91.,  ...,   72.,   73.,   72.],
          ...,
          [ 104.,  109.,  165.,  ...,  -92.,  -70.,  -48.],
          [  85.,  113.,  123.,  ...,  -90.,  -32.,   -9.],
          [  99.,   85.,   87.,  ...,   -4.,   39.,   49.]],

         [[  76.,   82.,   82.,  ...,   93.,   64.,   62.],
          [  95.,  110.,   93.,  ...,   71.,   73.,   86.],
          [ 121.,  114.,   73.,  ...,   96.,  123.,   95.],
          ...,
          [  90.,  106.,  123.,  ...,    8.,   61.

In [ ]:
# 回答　
%%time
# 2回目の実行　Wall time: 17.9 ms

print(dataset[-1000])  # 後ろから1000個目を使用

# getCtRawCandidate()関数の結果がキャッシュされているため、高速

(tensor([[[[ 113.,  103.,  103.,  ...,   94.,  106.,  104.],
          [  89.,   71.,   89.,  ...,   87.,   84.,   72.],
          [  90.,   65.,   58.,  ...,   68.,   68.,   93.],
          ...,
          [ 137.,  156.,  133.,  ..., -102., -113., -115.],
          [ 113.,  115.,  116.,  ..., -113.,  -93.,  -81.],
          [ 110.,  100.,  102.,  ..., -122.,  -73.,  -61.]],

         [[ 115.,  117.,   94.,  ...,   85.,   71.,   74.],
          [  76.,   71.,   80.,  ...,   56.,   51.,   58.],
          [  84.,   86.,   91.,  ...,   72.,   73.,   72.],
          ...,
          [ 104.,  109.,  165.,  ...,  -92.,  -70.,  -48.],
          [  85.,  113.,  123.,  ...,  -90.,  -32.,   -9.],
          [  99.,   85.,   87.,  ...,   -4.,   39.,   49.]],

         [[  76.,   82.,   82.,  ...,   93.,   64.,   62.],
          [  95.,  110.,   93.,  ...,   71.,   73.,   86.],
          [ 121.,  114.,   73.,  ...,   96.,  123.,   95.],
          ...,
          [  90.,  106.,  123.,  ...,    8.,   61.

## [2] __init__内でサンプルの配列をランダムにするようにLunaDatasetの実装を変更してください。そして、キャッシュをクリアし、変更したバージョンを実行してください。これにより、最初、及び2回目の実行時間にどのような影響がありますか。

In [ ]:
# 上部の「ランタイム」> 「ランタイムを出荷時設定にリセット」を行うことでキャッシュをクリアしてから実行　※[1]の内容は実行しないこと
# 回答（まずは1000個で試してみましょう）

In [ ]:
# 回答

# LunaDatasetの実装変更
!sed -i '169i\ \ \ \ \ \ \ \ random.shuffle(self.candidateInfo_list)' ./dsets.py
!sed -i '1iimport random' ./dsets.py



In [ ]:
# データセット

from dsets import LunaDataset
dataset = LunaDataset()

2020-11-03 08:42:02,430 INFO     pid:102 dsets:175:__init__ <dsets.LunaDataset object at 0x7f8b86307a58>: 56938 training samples


In [ ]:
# 1回目の実行　Wall time: 22min 25s
%%time

for i in range(1000):
    print(dataset[i])

# 結果がキャッシュされておらず、またサンプルの順番がランダムであることによりlru_cacheを活用できず（加えて、キャッシング回数が増加し）、
# [1]の（a）以上に長い実行時間になる。

ストリーミング出力は最後の 5000 行に切り捨てられました。
          [-925., -932., -919.,  ...,  138.,  150.,  185.],
          [-925., -933., -915.,  ...,  135.,  136.,  152.],
          ...,
          [-890., -928., -905.,  ..., -852., -878., -920.],
          [-890., -912., -858.,  ..., -871., -882., -907.],
          [-866., -899., -870.,  ..., -835., -783., -813.]],

         [[-792., -880., -912.,  ...,  131.,  142.,  154.],
          [-858., -896., -908.,  ...,  122.,  115.,  139.],
          [-855., -891., -902.,  ...,  116.,  109.,  148.],
          ...,
          [-888., -912., -913.,  ..., -880., -857., -885.],
          [-862., -892., -902.,  ..., -882., -849., -878.],
          [-849., -909., -922.,  ..., -735., -785., -848.]],

         [[-873., -932., -933.,  ...,  122.,  134.,  139.],
          [-901., -926., -921.,  ...,  101.,  135.,  152.],
          [-904., -909., -893.,  ...,  106.,  125.,  135.],
          ...,
          [-900., -914., -913.,  ..., -879., -885., -883.],
          [-901., -

In [ ]:
%%time
# 2回目の実行  Wall time: 7.83 s

for i in range(1000):
    print(dataset[i])

# ランダム化の影響でgetCt()関数のlru_cacheは活用できなくなるものの、
# getCtRawCandidate()関数の結果がキャッシュされているため、[1]のbと同程度の実行時間になる。

ストリーミング出力は最後の 5000 行に切り捨てられました。
          [-925., -932., -919.,  ...,  138.,  150.,  185.],
          [-925., -933., -915.,  ...,  135.,  136.,  152.],
          ...,
          [-890., -928., -905.,  ..., -852., -878., -920.],
          [-890., -912., -858.,  ..., -871., -882., -907.],
          [-866., -899., -870.,  ..., -835., -783., -813.]],

         [[-792., -880., -912.,  ...,  131.,  142.,  154.],
          [-858., -896., -908.,  ...,  122.,  115.,  139.],
          [-855., -891., -902.,  ...,  116.,  109.,  148.],
          ...,
          [-888., -912., -913.,  ..., -880., -857., -885.],
          [-862., -892., -902.,  ..., -882., -849., -878.],
          [-849., -909., -922.,  ..., -735., -785., -848.]],

         [[-873., -932., -933.,  ...,  122.,  134.,  139.],
          [-901., -926., -921.,  ...,  101.,  135.,  152.],
          [-904., -909., -893.,  ...,  106.,  125.,  135.],
          ...,
          [-900., -914., -913.,  ..., -879., -885., -883.],
          [-901., -

## [3] ランダム化を戻し、getCtの@functools.lru_cache(1, typed=True)デコレーターをコメントアウトしてください。キャッシュをクリアし、変更したバージョンを実行してください。実行時間はどのように変わりますか？ 

In [ ]:
# 上部の「ランタイム」> 「ランタイムを出荷時設定にリセット」を行うことでキャッシュをクリアしてから実行　※[1][2]の内容は実行しないこと
# 回答（まずは1000個で試してみましょう）

In [ ]:
# 回答　
# コードを修正
!sed -i -e '/@functools.lru_cache(1, typed=True)/d' ./dsets.py


In [ ]:
from dsets import LunaDataset
dataset = LunaDataset()


2020-11-03 11:19:57,480 INFO     pid:103 dsets:172:__init__ <dsets.LunaDataset object at 0x7f9b376189b0>: 56938 training samples


In [ ]:
%%time

#実行　Wall time: 6min 49s

for i in range(1000):
    print(dataset[i])

# 初回実行でも、getCt()関数のlru_cacheを使用しなくなった結果、[1]のa以上に長い実行時間になる。

ストリーミング出力は最後の 5000 行に切り捨てられました。
          [  66.,  107.,   96.,  ...,   48.,   13.,   15.],
          [  73.,  104.,  135.,  ...,   59.,   39.,   13.],
          ...,
          [-783., -783., -806.,  ..., -857., -846., -804.],
          [-770., -831., -843.,  ..., -870., -858., -857.],
          [-855., -878., -836.,  ..., -854., -869., -867.]],

         [[  77.,   80.,  108.,  ...,   32.,   58.,   50.],
          [  47.,  129.,  145.,  ...,   23.,   -9.,    8.],
          [  92.,  128.,  107.,  ...,   74.,    1.,   12.],
          ...,
          [-839., -839., -843.,  ..., -827., -818., -805.],
          [-798., -768., -798.,  ..., -842., -875., -862.],
          [-808., -800., -831.,  ..., -840., -868., -862.]],

         [[  86.,   88.,  104.,  ...,   -9.,   32.,   84.],
          [  83.,   71.,   61.,  ...,   45.,   76.,   70.],
          [  50.,   80.,  104.,  ...,  111.,   71.,   36.],
          ...,
          [-810., -737., -785.,  ..., -816., -835., -863.],
          [-833., -